In [66]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model
import pickle

In [67]:
# Load the tokenizer
with open('tokenizer.pkl', 'rb') as f:
    word_index = pickle.load(f)

reverse_word_index = {value: key for key, value in word_index.items()}

In [68]:
# Loading the pre-trained model with relu activation function 

model = load_model('SimpleRnn_imdb.h5')
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_4 (SimpleRNN)        │ (None, 500, 128)       │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 500, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_5 (SimpleRNN)        │ (None, 64)             │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,325,315 (5.06 MB)

 Trainable params: 1,325,313 (5.06 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [69]:
model.get_weights()

[array([[-0.02176417,  0.00292836, -0.03358303, ..., -0.00221231,
         -0.02517314, -0.00182923],
        [ 0.02217165, -0.0048374 ,  0.01360605, ...,  0.00791165,
         -0.0034124 , -0.01517934],
        [ 0.03419331, -0.03407732,  0.00209415, ..., -0.00259004,
         -0.03489134, -0.00326097],
        ...,
        [-0.05803373, -0.02129197,  0.0159745 , ..., -0.02593427,
          0.03086851, -0.02700802],
        [ 0.05797685, -0.03743098,  0.04462858, ...,  0.04305399,
         -0.039929  , -0.00576007],
        [-0.00511815, -0.04570761,  0.0261557 , ...,  0.00968209,
         -0.0209402 ,  0.03235263]], dtype=float32),
 array([[ 0.09757566, -0.08057367,  0.1619149 , ...,  0.06136416,
          0.03297292, -0.11027479],
        [-0.10753874,  0.09411561,  0.03778527, ...,  0.07493274,
          0.11992583,  0.03188687],
        [-0.01830975, -0.03923976, -0.00591082, ...,  0.13814153,
          0.01354577, -0.0030879 ],
        ...,
        [-0.07170143,  0.08324607, -0.0

Helper Functions

In [70]:
# Function to decode reviews
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

# Function to preprocess user input
def preprocess_text(text):
    words = text.lower().split()
    encoded_review = [word_index.get(word, 2) + 3 for word in words]
    padded_review = sequence.pad_sequences([encoded_review], maxlen=500)  # Match training input length
    return padded_review

In [71]:
def predict_sentiment(review):
    preprocessed_input = preprocess_text(review)
    prediction = model.predict(preprocessed_input)[0][0]  # Extract scalar value

    # Define thresholds for classification
    if prediction >= 0.6:
        sentiment = 'Positive'
    elif prediction <= 0.4:
        sentiment = 'Negative'
    else:
        sentiment = 'Neutral'

    return sentiment, prediction

User Input And Prediction

In [75]:
test_reviews = [
    "The movie had some good moments, but overall, it was just okay.",
    "Not bad, but not great either. A decent watch for a lazy afternoon.",
    "There were parts I enjoyed, but the ending was disappointing.",
    "It had a good concept, but the execution could have been better.",
    "Some scenes were good, but overall, it felt a bit underwhelming."
]




# Running Predictions
for review in test_reviews:
    sentiment, score = predict_sentiment(review)
    print(f'Review: {review}')
    print(f'Sentiment: {sentiment} | Prediction Score: {score:.4f}\n')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Review: The movie had some good moments, but overall, it was just okay.
Sentiment: Positive | Prediction Score: 0.6006

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Review: Not bad, but not great either. A decent watch for a lazy afternoon.
Sentiment: Neutral | Prediction Score: 0.5389

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Review: There were parts I enjoyed, but the ending was disappointing.
Sentiment: Neutral | Prediction Score: 0.5614

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Review: It had a good concept, but the execution could have been better.
Sentiment: Neutral | Prediction Score: 0.4590

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Review: Some scenes were good, but overall, it felt a bit underwhelming.
Sentiment: Neutral | Prediction Score: 0.5762

